In [1]:
# imports
import requests
import pandas as pd
import os
import json

# Foursquare

## 1. Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

#### Below is the code to fetch details about restaurants around the bike stations collected in Part1 (i.e. latitude and longitude data of bike stations from citybikes API)

In [2]:

# To get Latitude,Longitude input from stations_df created in Part1 using citybikes API
stations_df = pd.read_csv('../data/citybikes_stations.csv')
    
# Extract latitude and longitude as separate lists
latitude_list = stations_df['Latitude'].tolist()
longitude_list = stations_df['Longitude'].tolist()

#Taking coordinates as tuple to use as input for foursquare
coordinates_list = list(zip(latitude_list, longitude_list))

#Restricting number of stations
first_30_coordinates=coordinates_list[:30]

#parameters to make an API call to Foursquare
FOURSQUARE_KEY = os.getenv('API_KEY')

# Foursquare API endpoint URL for exploring venues
url = 'https://api.foursquare.com/v3/places/search'
headers = {"Accept": "application/json"}
headers['Authorization'] = FOURSQUARE_KEY

# List to store venue data
venues_data_list = []

# Iterate over the list of coordinates (latitude, longitude tuples)
for latitude, longitude in first_30_coordinates:
    # Make a new request for venues around each bike station
    venues_params = {
        'll': f'{latitude},{longitude}',  # Latitude and longitude combined as a string
        'radius': 1000,
        'limit': 50,  # Number of venues to retrieve (adjust as needed)
        'categories': 13065  # category code for restaurants in Foursquare
    }

    # Make the API request for venues
    venues_response = requests.get(url, params=venues_params, headers=headers)

    # Check if the venues response is successful
    if venues_response.status_code == 200:
        venues_data = json.loads(venues_response.text)

        for venue in venues_data.get('results', []):
            # Extract category name from categories list
            categories = venue.get('categories', [])
            category_name = ', '.join([cat.get('name') for cat in categories])

            venue_details = {
                'Name': venue.get('name'),  # Venue name
                'Category': category_name,  # Category name(s) as a comma-separated string
                #'Latitude': venue.get('geocodes', {}).get('main', {}).get('latitude'),
                #'Longitude': venue.get('geocodes', {}).get('main', {}).get('longitude'),
                'Closed Bucket': venue.get('closed_bucket'),
                'Distance': venue.get('distance'),
                'Latitude':latitude,
                'Longitude':longitude
                # Add more keys as needed
            }
            venues_data_list.append(venue_details)

# venues_data_list now contains venue details for each station


# Create a DataFrame from the venues data list
venues_df = pd.DataFrame(venues_data_list)

# Print the DataFrame
venues_df


,Name,Category,Closed Bucket,Distance,Latitude,Longitude
0,Béchu,"Bakery, Cafe, Coffee, and Tea House, Restaurant",LikelyOpen,487,48.865983,2.275725
1,Restaurant Brach,Restaurant,LikelyOpen,528,48.865983,2.275725
2,Le Flandrin,"Gastropub, Brasserie",VeryLikelyOpen,324,48.865983,2.275725
3,Le Zinc du 16,"Café, Restaurant",VeryLikelyOpen,246,48.865983,2.275725
4,Le Petit Rétro,"Bistro, Pizzeria, French Restaurant",LikelyOpen,736,48.865983,2.275725
...,...,...,...,...,...,...
1457,Le Perchoir Porte de Versailles,"Cocktail Bar, Restaurant",VeryLikelyOpen,730,48.833101,2.299380
1458,Zambalha,Tibetan Restaurant,VeryLikelyOpen,753,48.833101,2.299380
1459,Bonzai,"Fast Food Restaurant, Japanese Restaurant",LikelyOpen,771,48.833101,2.299380
1460,KFC,"Fast Food Restaurant, Fried Chicken Joint",LikelyOpen,487,48.833101,2.299380


## 2. Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

#### I have taken Latitude,Longitude values from citybikes API as input for foursquare API

## 3.Put your parsed results into a DataFrame

#### venues_df = pd.DataFrame(venues_data_list)

# Yelp

## 1.Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

#### Below is the code to fetch details about restaurants around the bike stations collected in Part1 (i.e. latitude and longitude data of bike stations from citybikes API)

In [3]:
# To get Latitude,Longitude input from stations_df
stations_df = pd.read_csv('../data/citybikes_stations.csv')

# Extract latitude and longitude as separate lists
latitude_list = stations_df['Latitude'].tolist()
longitude_list = stations_df['Longitude'].tolist()

#Restricting number of stations
coordinates_list = list(zip(latitude_list, longitude_list))
first_30_coordinates=coordinates_list[:30]


YELP_API_KEY = os.getenv('YELP_API')
API_ENDPOINT = 'https://api.yelp.com/v3/businesses/search'

# Yelp API endpoint URL for exploring restaurants
yelp_url = 'https://api.yelp.com/v3/businesses/search'
yelp_headers = {
    "Authorization": f"Bearer {YELP_API_KEY}"
}

# List to store venue data
venues_data_list = []

# Make a request to Yelp API to get restaurants around each bike station
# Iterate over the list of coordinates (latitude, longitude tuples)

for latitude, longitude in first_30_coordinates:
    # Make a new request for venues around each bike station
        yelp_params = {
            'latitude': latitude,
            'longitude': longitude,
            'radius': 1000,  # Radius for restaurant search (adjust as needed)
            'categories': 'restaurants',  
            'limit': 50,  # Number of restaurants to retrieve (adjust as needed)
            'attributes': 'parking_bike,restaurants_takeout,liked_by_vegetarians,wifi_free',
            'open_now': True
        }

        yelp_response = requests.get(API_ENDPOINT, params=yelp_params, headers=yelp_headers)

        # Check if the Yelp response is successful
        if yelp_response.status_code == 200:
            yelp_data = json.loads(yelp_response.text)
            for business in yelp_data.get('businesses', []):
                categories = [category.get('title') for category in business.get('categories', [])]
                address1 = business.get('location', {}).get('address1')
                city = business.get('location', {}).get('city')
                if address1 and city:
                    address = ', '.join([address1, city])
                else:
                    address = 'N/A'  # You can set a default value or handle it in your own way
                restaurant_details = {
                'Name': business.get('name'),
                'Categories': ', '.join(categories),
                'Rating': business.get('rating'),
                'Address': address,
                'Res_Latitude': business.get('coordinates', {}).get('latitude'),
                'Res_Longitude': business.get('coordinates', {}).get('longitude'),
                'Reviews': business.get('review_count'),
                'Distance': business.get('distance'),
                'Phone': business.get('phone'),
                'Closed':business.get('is_closed'),
                'Price':business.get('price'),
                'Latitude':latitude,
                'Longitude':longitude
                }
                venues_data_list.append(restaurant_details)
        
# Create a DataFrame from the restaurant data list
restaurants_df = pd.DataFrame(venues_data_list)

# Print the DataFrame
print(restaurants_df)


                      Name                              Categories  Rating  \
0           Le Victor Hugo               Brasseries, French, Cafes     3.5   
1            Honest Lawyer           Pubs, Sports Bars, Gastropubs     3.5   
2             Le Frankl'in                      French, Brasseries     3.0   
3            I love Bo-Bun                              Vietnamese     2.5   
4          Au Petit Suisse                      Brasseries, French     4.0   
..                     ...                                     ...     ...   
371   Le Cosy Montparnasse        Jazz & Blues, Piano Bars, French     3.5   
372           Le Cochinous                            Bars, French     3.0   
373  Le Comptoir de Sèvres                        Brasseries, Bars     3.0   
374      Dupont Versailles        Brasseries, Coffee & Tea, French     2.5   
375           Le Diplomate  Tobacco Shops, Sports Bars, Brasseries     3.5   

                                  Address  Res_Latitude  Res_Lo

## 2.Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

#### I have used below two filters in yelp_parameters:

attributes': 'parking_bike,restaurants_takeout,liked_by_vegetarians,wifi_free',
'open_now': True

## 3.Put your parsed results into a DataFrame

#### restaurants_df = pd.DataFrame(venues_data_list)

# Comparing Results

### Which API provided you with more complete data? Provide an explanation. 

#### Yelp provided more information as it gave reviews,rating,price and phone number for restaurants. Also you can use may filetrs in parameters while requaeting API.

#### Get the top 10 restaurants according to their rating

In [4]:
Top_10_Restaurants=restaurants_df.sort_values('Rating',ascending=False) 

In [5]:
Top_10_Restaurants=Top_10_Restaurants[:10]

In [6]:
Top_10_Restaurants.sort_values('Reviews',ascending=False)

,Name,Categories,Rating,Address,Res_Latitude,Res_Longitude,Reviews,Distance,Phone,Closed,Price,Latitude,Longitude
295,Pedzouille,"Bistros, Steakhouses",4.5,"66 rue du Faubourg Poissonnière, Paris",48.876133,2.348528,25,531.243022,+33986404242,False,€€,48.879089,2.354235
29,Mardin Çorba Salonu,"Turkish, Middle Eastern, Soup",4.5,"19 rue du Faubourg-Saint-Denis, Paris",48.870440,2.352900,10,927.414924,,False,€,48.867872,2.364898
180,Le Lafon,"Brasseries, Cafes",4.5,"21 rue Réaumur, Paris",48.865069,2.357294,3,925.541809,+33157409378,False,None,48.871044,2.366104
159,Eric Kayser,"Bakeries, Delicatessen, Sandwiches",4.5,"61bis avenue de la Grande-Armée, Paris",48.875840,2.287420,2,792.069079,+33145005261,False,None,48.882878,2.287667
138,Casa di Panini,"Creperies, Fast Food",5.0,"17 rue Jussieu, Paris",48.845772,2.355450,1,1070.568306,+33950093020,False,None,48.851519,2.343670
20,Pomme de Pain,"Fast Food, Sandwiches",5.0,"101 rue Berger, Paris",48.860590,2.349000,1,1113.460099,+33172636455,False,None,48.853756,2.339096
218,Le Ventadour,Brasseries,5.0,"46 rue des Petits Champs, Paris",48.867390,2.334450,1,889.254866,+33142965298,False,None,48.874423,2.328469
339,Casa di Panini,"Creperies, Fast Food",5.0,"17 rue Jussieu, Paris",48.845772,2.355450,1,438.425519,+33950093020,False,None,48.845047,2.349465
261,Casa di Panini,"Creperies, Fast Food",5.0,"17 rue Jussieu, Paris",48.845772,2.355450,1,322.098004,+33950093020,False,None,48.843893,2.351966
66,Pomme de Pain,"Fast Food, Sandwiches",5.0,"101 rue Berger, Paris",48.860590,2.349000,1,1115.488023,+33172636455,False,None,48.856452,2.334852


## Note: Data Cleaning details have been added in separate notebook 
